In [38]:
import pandas as pd

In [39]:
df = pd.read_csv("combined_results.csv")
df = df[df["split_method"] != "random_split"]

In [ ]:
metric = "RMSE"
dfm   = df[df["metric"] == metric]

avg_per_task = dfm.groupby("task_id")["value"].mean()

avg_per_task = avg_per_task.reset_index(name=f"avg_{metric}")
avg_per_task.to_csv(f"avg_{metric}_per_task.csv", index=False)


Average Difference and Average Relative Difference RMSE

In [40]:
metric = "RMSE"
dfm   = df[df["metric"] == metric]

pivot_rmse = dfm.pivot_table(
    index="task_id",
    columns="model",
    values="value",
    aggfunc="mean"
)

pivot_rmse.to_csv(f"avg_{metric}_per_task_per_model.csv", float_format="%.3f")


In [41]:
def avg_relative_diff(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:

    if error_metric:
        best_per_task = pivot.min(axis=1)   # best is minimum
        # (v - best)/best * 100
        rel_diff = (pivot.sub(best_per_task, axis=0)
                         .div(best_per_task, axis=0)
                         * 100)
    else:
        best_per_task = pivot.max(axis=1)   # best is maximum
        # (best - v)/best * 100
        rel_diff = (best_per_task.sub(pivot, axis=0)
                              .div(best_per_task, axis=0)
                              * 100)

    # average across tasks (skipping any NaNs)
    return rel_diff.mean(axis=0)


avg_diff_rmse = avg_relative_diff(pivot_rmse, error_metric=True)

print("Average relative difference (RMSE):")
print(avg_diff_rmse.round(2))


Average relative difference (RMSE):
model
ConstantPredictor          280.01
Engression                  78.69
FTTransformer               43.66
LGBMRegressor               35.15
LinearRegressor          92264.08
MLP                        133.95
RandomForestRegressor       39.83
ResNet                     109.58
dtype: float64


In [42]:
def normalized_accuracy(pivot: pd.DataFrame,
                        error_metric: bool = True) -> pd.Series:

    if error_metric:
        best = pivot.min(axis=1)  

        mid  = pivot.apply(
            lambda s: s.dropna().nlargest(3).min(),
            axis=1
        )
        norm = (
            pivot
            .rsub(mid,       axis=0)    # mid[t] - pivot.loc[t, m]
            .div(mid - best, axis=0)
            .clip(0, 1)
        )
    else:
        best = pivot.max(axis=1)
        mid  = pivot.apply(
            lambda s: s.dropna().nsmallest(3).max(),
            axis=1
        )
        # 3) norm_acc per cell = (acc - mid) / (best - mid), clipped
        norm = (
            pivot
            .rsub(mid,          axis=0)   # pivot.loc[t,m] - mid[t]
            .mul(-1)
            .div(mid - best, axis=0)
            .clip(0, 1)
        )

    return norm.mean(axis=0)


avg_norm_acc_rmse = normalized_accuracy(pivot_rmse, error_metric=True)
print("Average normalized accuracy (from RMSE):")
print((100 * avg_norm_acc_rmse).round(2))


Average normalized accuracy (from RMSE):
model
ConstantPredictor         8.78
Engression               49.41
FTTransformer            64.33
LGBMRegressor            79.00
LinearRegressor           4.04
MLP                      47.25
RandomForestRegressor    60.65
ResNet                   35.17
dtype: float64


In [45]:
def avg_rank(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:
  
    ranks = pivot.rank(
        axis=1,
        method="average",   
        ascending=error_metric
    )

    return ranks.mean(axis=0)

avg_rank_rmse = avg_rank(pivot_rmse, error_metric=True)
print("Average rank (RMSE):")
print(avg_rank_rmse.round(2))



Average rank (RMSE):
model
ConstantPredictor        6.69
Engression               3.97
FTTransformer            3.29
LGBMRegressor            2.47
LinearRegressor          6.39
MLP                      4.44
RandomForestRegressor    3.19
ResNet                   4.62
dtype: float64


Average Difference and Average Relative Difference CRPS

In [46]:
metric1 = "CRPS"
dfm1   = df[df["metric"] == metric1]

pivot_crps = dfm1.pivot_table(
    index="task_id",
    columns="model",
    values="value",
    aggfunc="mean"
)

# 4) (Optional) write out to CSV
pivot_crps.to_csv(f"avg_{metric1}_per_task_per_model.csv", float_format="%.3f")


In [48]:
def avg_relative_diff(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:

    if error_metric:
        best_per_task = pivot.min(axis=1)   # best is minimum
        # (v - best)/best * 100
        rel_diff = (pivot.sub(best_per_task, axis=0)
                         .div(best_per_task, axis=0)
                         * 100)
    else:
        best_per_task = pivot.max(axis=1)   # best is maximum
        # (best - v)/best * 100
        rel_diff = (best_per_task.sub(pivot, axis=0)
                              .div(best_per_task, axis=0)
                              * 100)

    # average across tasks (skipping any NaNs)
    return rel_diff.mean(axis=0)


avg_diff_crps = avg_relative_diff(pivot_crps, error_metric=True)

print("Average relative difference (CRPS):")
print(avg_diff_crps.round(2))


Average relative difference (CRPS):
model
ConstantPredictor          371.53
DGBT                        42.65
DRF                         62.20
Engression               15127.91
FTTransformer               46.80
LGBMRegressor               42.55
LinearRegressor          71983.54
MLP                         54.37
RandomForestRegressor       50.31
ResNet                     123.53
dtype: float64


In [26]:
def normalized_accuracy(pivot: pd.DataFrame,
                        error_metric: bool = True) -> pd.Series:

    if error_metric:
        best = pivot.min(axis=1)  

        mid  = pivot.apply(
            lambda s: s.dropna().nlargest(3).min(),
            axis=1
        )
        norm = (
            pivot
            .rsub(mid,       axis=0)    # mid[t] - pivot.loc[t, m]
            .div(mid - best, axis=0)
            .clip(0, 1)
        )
    else:
        best = pivot.max(axis=1)
        mid  = pivot.apply(
            lambda s: s.dropna().nsmallest(3).max(),
            axis=1
        )
        # 3) norm_acc per cell = (acc - mid) / (best - mid), clipped
        norm = (
            pivot
            .rsub(mid,          axis=0)   # pivot.loc[t,m] - mid[t]
            .mul(-1)
            .div(mid - best, axis=0)
            .clip(0, 1)
        )

    return norm.mean(axis=0)


avg_norm_acc_crps = normalized_accuracy(pivot_crps, error_metric=True)
print("Average normalized accuracy (from CRPS):")
print((100 * avg_norm_acc_crps).round(2))


Average normalized accuracy (from CRPS):
model
ConstantPredictor         5.78
DGBT                     63.41
DRF                      60.20
Engression               55.72
FTTransformer            55.37
LGBMRegressor            60.78
LinearRegressor           6.94
MLP                      33.37
RandomForestRegressor    46.11
ResNet                   28.84
dtype: float64


In [47]:
def avg_rank(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:
  
    ranks = pivot.rank(
        axis=1,
        method="average",   
        ascending=error_metric
    )

    return ranks.mean(axis=0)

avg_rank_crps = avg_rank(pivot_crps, error_metric=True)
print("Average rank (CRPS):")
print(avg_rank_crps.round(2))



Average rank (CRPS):
model
ConstantPredictor        8.89
DGBT                     3.25
DRF                      4.06
Engression               4.56
FTTransformer            4.42
LGBMRegressor            3.61
LinearRegressor          8.11
MLP                      6.12
RandomForestRegressor    4.78
ResNet                   6.03
dtype: float64


Average Difference and Average Relative Difference LOGLOSS

In [ ]:
metric2 = "LogLoss"
dfm2   = df[df["metric"] == metric2]

pivot_ll = dfm2.pivot_table(
    index="task_id",
    columns="model",
    values="value",
    aggfunc="mean"
)

pivot_ll.to_csv(f"avg_{metric2}_per_task_per_model.csv", float_format="%.3f")


In [ ]:
def avg_relative_diff(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:

    if error_metric:
        best_per_task = pivot.min(axis=1)   # best is minimum
        # (v - best)/best * 100
        rel_diff = (pivot.sub(best_per_task, axis=0)
                         .div(best_per_task, axis=0)
                         * 100)
    else:
        best_per_task = pivot.max(axis=1)   # best is maximum
        # (best - v)/best * 100
        rel_diff = (best_per_task.sub(pivot, axis=0)
                              .div(best_per_task, axis=0)
                              * 100)

    return rel_diff.mean(axis=0)


avg_diff_ll = avg_relative_diff(pivot_ll, error_metric=True)

print("Average relative difference (LogLoss):")
print(avg_diff_ll.round(2))


Average relative difference (LogLoss):
model
ConstantPredictor          371.53
DGBT                        42.65
DRF                         62.20
Engression               15127.91
FTTransformer               46.80
LGBMRegressor               42.55
LinearRegressor          71983.54
MLP                         54.37
RandomForestRegressor       50.31
ResNet                     123.53
dtype: float64


In [19]:
def normalized_accuracy(pivot: pd.DataFrame,
                        error_metric: bool = True) -> pd.Series:

    if error_metric:
        best = pivot.min(axis=1)  

        mid  = pivot.apply(
            lambda s: s.dropna().nlargest(3).min(),
            axis=1
        )
        norm = (
            pivot
            .rsub(mid,       axis=0)    # mid[t] - pivot.loc[t, m]
            .div(mid - best, axis=0)
            .clip(0, 1)
        )
    else:
        best = pivot.max(axis=1)
        mid  = pivot.apply(
            lambda s: s.dropna().nsmallest(3).max(),
            axis=1
        )
        # 3) norm_acc per cell = (acc - mid) / (best - mid), clipped
        norm = (
            pivot
            .rsub(mid,          axis=0)   # pivot.loc[t,m] - mid[t]
            .mul(-1)
            .div(mid - best, axis=0)
            .clip(0, 1)
        )

    return norm.mean(axis=0)


avg_norm_acc_ll = normalized_accuracy(pivot_ll, error_metric=True)
print("Average normalized accuracy (from LogLoss):")
print((100 * avg_norm_acc_ll).round(2))


Average normalized accuracy (from LogLoss):
model
ConstantPredictor          0.00
Engression                36.68
FTTransformer             83.33
LGBMClassifier            15.88
LogisticRegressor         42.64
MLP                       74.85
RandomForestClassifier    15.88
ResNet                    72.90
dtype: float64


In [49]:
def avg_rank(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:
  
    ranks = pivot.rank(
        axis=1,
        method="average",   
        ascending=error_metric
    )

    return ranks.mean(axis=0)

avg_rank_ll = avg_rank(pivot_ll, error_metric=True)
print("Average rank (LogLoss):")
print(avg_rank_ll.round(2))



Average rank (LogLoss):
model
ConstantPredictor         7.96
Engression                4.52
FTTransformer             2.27
LGBMClassifier            5.41
LogisticRegressor         4.48
MLP                       2.83
RandomForestClassifier    5.41
ResNet                    2.87
dtype: float64


Average Difference and Average Relative Difference Accuracy

In [50]:
metric3 = "Accuracy"
dfm3   = df[df["metric"] == metric3]

pivot_acc = dfm3.pivot_table(
    index="task_id",
    columns="model",
    values="value",
    aggfunc="mean"
)

pivot_acc.to_csv(f"avg_{metric3}_per_task_per_model.csv", float_format="%.5f")


In [ ]:
def avg_relative_diff(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:

    if error_metric:
        best_per_task = pivot.min(axis=1)   # best is minimum
        # (v - best)/best * 100
        rel_diff = (pivot.sub(best_per_task, axis=0)
                         .div(best_per_task, axis=0)
                         * 100)
    else:
        best_per_task = pivot.max(axis=1)   # best is maximum
        # (best - v)/best * 100
        rel_diff = (pivot.sub(best_per_task, axis=0)     
                         .mul(-1)                         
                         .div(best_per_task , axis=0)
                         * 100)

    return rel_diff.mean(axis=0)


avg_diff_accuracy = avg_relative_diff(pivot_acc, error_metric=False)

print("\nAverage relative difference (Accuracy):")
print(avg_diff_accuracy.round(2))



Average relative difference (Accuracy):
model
ConstantPredictor         45.44
Engression                17.32
FTTransformer              2.49
LGBMClassifier             0.36
LogisticRegressor          8.46
MLP                        3.56
RandomForestClassifier     1.27
ResNet                     3.35
dtype: float64


In [37]:
def normalized_accuracy(pivot: pd.DataFrame,
                        error_metric: bool = True) -> pd.Series:

    if error_metric:
        best = pivot.min(axis=1)  

        mid  = pivot.apply(
            lambda s: s.dropna().nlargest(3).min(),
            axis=1
        )
        norm = (
            pivot
            .rsub(mid,       axis=0)    # mid[t] - pivot.loc[t, m]
            .div(mid - best, axis=0)
            .clip(0, 1)
        )
    else:
        best = pivot.max(axis=1)
        mid  = pivot.apply(
            lambda s: s.dropna().nsmallest(3).max(),
            axis=1
        )
        # 3) norm_acc per cell = (acc - mid) / (best - mid), clipped
        norm = (
            pivot
            .rsub(mid,          axis=0)   # pivot.loc[t,m] - mid[t]
            .div(mid - best, axis=0)
            .clip(0, 1)
        )

    return norm.mean(axis=0)


avg_norm_acc_acc = normalized_accuracy(pivot_acc, error_metric=False)
print("Average normalized accuracy (from Accuracy):")
print((100 * avg_norm_acc_acc).round(3))


Average normalized accuracy (from Accuracy):
model
ConstantPredictor          0.000
Engression                 2.183
FTTransformer             62.432
LGBMClassifier            93.086
LogisticRegressor          9.558
MLP                       48.648
RandomForestClassifier    81.832
ResNet                    53.322
dtype: float64


In [51]:
def avg_rank(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:
  
    ranks = pivot.rank(
        axis=1,
        method="average",   
        ascending=error_metric
    )

    return ranks.mean(axis=0)

avg_rank_acc = avg_rank(pivot_acc, error_metric=False)
print("Average rank (Accuracy):")
print(avg_rank_acc.round(2))



Average rank (Accuracy):
model
ConstantPredictor         7.96
Engression                6.70
FTTransformer             3.32
LGBMClassifier            1.78
LogisticRegressor         5.70
MLP                       4.00
RandomForestClassifier    2.48
ResNet                    3.87
dtype: float64
